In [20]:
import os
print(os.getcwd())
# change working directory to project root
import platform
if platform.system() is "Windows":
    os.chdir('E:\\sven\\git\\cardio')
else:
    os.chdir('/Users/minority/Code/Git/cardio')
print(os.getcwd())
from src.utils.utils_io import Console_and_file_logger, ensure_dir
from src.utils.file_wrapper.app.Files import Basefile, Dicomfile
from src.utils.file_wrapper.app.IO import FileWalker
import logging
Console_and_file_logger('extract_segmentations_10', logging.INFO)

import re
#import nrrd
import numpy as np
from scipy import stats
from collections import OrderedDict
import json
import SimpleITK as sitk
from bs4 import BeautifulSoup
from bs4 import SoupStrainer
from collections import Counter
import cv2


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# specific imports
import xml.etree.ElementTree as ET
import glob

E:\sven\git\cardio
E:\sven\git\cardio


In [3]:
class Contour():
    
    def __init__(self, uid, image_size, pixel_size, points, sub_pixel_res, tag):
        self.uid = uid
        self.image_size = image_size
        self.pixel_size = pixel_size
        self.points = points
        self.sub_pixel_res = sub_pixel_res
        self.tag = tag
        
    uid = 0  
    image_size = (0,0)
    pixel_size = (0,0)
    sub_pixel_res = 0
    points = []

In [4]:
def contour_factory(contours, tag):
    """
    Expects a list of segmentation-elements
    Reads/collects all neccessary data for each contour-element
    Wraps each contour information in a contour-object,
    builds a list of Contours and returns them
    Params: list of contour-xml tags:
    e.g. call:
    xml_file = 'D:\\git\\cardio\\data\\processed\\Segmentations\\0AE4R74L190001011238291.3.12.2.1107.5.99.2.1013.30000008030612131778100072179.cvi42wsx'
    with open(xml_file) as fp:
    soup = BeautifulSoup(fp, 'lxml')
    saendocardialContour = soup.find_all('hash:item', {'hash:key' : 'saendocardialContour'})
    
    """
    cont = {}
    logging.info('building {} contours'.format(len(contours)))
    for contour in contours:
        #print(contour)
        uid = contour.parent.parent.attrs.get('hash:key')
        image_width = contour.find('hash:item', {'hash:key' : 'ImageSize'}).find('size:width').text
        image_height = contour.find('hash:item', {'hash:key' : 'ImageSize'}).find('size:height').text
        pixel_width = contour.find('hash:item', {'hash:key' : 'PixelSize'}).find('size:width').text
        pixel_height = contour.find('hash:item', {'hash:key' : 'PixelSize'}).find('size:height').text
        sub_pixel = contour.find('hash:item', {'hash:key' : 'SubpixelResolution'}).text
        points_t = contour.find('hash:item', {'hash:key' : 'Points'})
        points_x = [point.text for point in points_t.find_all('point:x')]
        points_x[:] = [int(int(x) / int(sub_pixel)) for x in points_x]
        points_y = [point.text for point in points_t.find_all('point:y')]
        points_y[:] = [int(int(y) / int(sub_pixel)) for y in points_y]
        points = list(zip(points_x, points_y))
        
        
        # create a Contour class for each contour
        cont[str(uid)] = Contour(uid, 
                                     (image_width, image_height), 
                                     (pixel_width, pixel_height),
                                    points,
                                    sub_pixel, tag)
    return cont

In [5]:
def extract_contours(soup):
    """
    Expects the Segmentation-XML-File as Beautifulsup-object
    Returns: a dictionary with segmentation-tag : list of segmentation-tag elements
    """
    c = {}
    contours = {'saendocardialContour': [],
           'saepicardialContour' : [],
           'sarvendocardialContour': [],
           'sarvepicardialContour': []}
    
    # Find root elements of each contour tag
    for tag, contour_elements in contours.items():
        logging.info('read: {}'.format(tag))
        contours[tag] = soup.find_all('hash:item', {'hash:key' : tag})
        logging.info('found: {} contours'.format(len(contours[tag])))
        
    
    """
    convert to contour-objects
    Expects a dictionary object with:
    segmentation-tag : list of segmentation-tag elements
    converts the list of segmentation-elements
    into a flat dictionary with {uid : [contour]}
    """

    for tag, contour_elements in contours.items():
        if len(contour_elements) > 0:
            logging.info('extract all {} segmentations'.format(tag))
            # transform all contour elements in contour objects
            contours = contour_factory(contour_elements, tag)
            # update contours dictionary
            # we might have multiple contours per slice
            for uid, cont in contours.items():      
                if uid in c.keys():
                    c[uid].append(cont)
                else:
                    c[uid] = [cont]

    return c


In [6]:
class GetSegmenationDicoms(Dicomfile):
    """
    Used to filter all dicom images which are part of the segmentation 
    by comparing the series id with the valid series ids injected by the filewalker
    save them to dest
    returns sitk-image
    """

    def action(self):
        
        if self.is_part_of_volume():
            # save dicom to destination path, keep all meta information
            #self.save(self.destination)
            return self
    
    def is_part_of_volume(self):
        """
        check if series-id is in valid series ids
        valid series ids are capturedinjected by filewalker to the dicomwrapper class
        and available via self.args
        """
        series_id = self.get_metadata_maybe('0020|000e')
        if series_id in self.args:
            return True
        else:
            return False
    
    def merge(self, a, b):
        if isinstance(a, dict):
            a = []
            a.append(b)
        else:
            a.append(b)
        return a
        
    
        

In [7]:
class GetDicomSeriesIDs(Dicomfile):
    """
    check if dicom file is part of the segmentation
    all valid segmentations are injected by the filewalker
    returns the series id of this sop
    """
    
    def action(self):
        if self.is_segment():
            #return the series-id
            series_id = self.get_metadata_maybe('0020|000e')
            return series_id
    
    def is_segment(self):
        # check if dicom-series is in contour-dict (injected by filewalker)
        uid = self.get_metadata_maybe('0008|0018')
        if uid in self.args.keys():
            return True
        else:
            return False
    
    def merge(self, a, b):
        if isinstance(a, dict):
            a = []
            a.append(b)
        else:
            a.append(b)
        return a

In [8]:
def sort_dicoms(dicom_images, sort_for_time=True):
    
    """
    Sort all slices first after time, then origin
    """
    
    logging.debug('images to sort: {}'.format(len(dicom_images)))  

    def origin_sort(dicom_wrapper):
        origin = dicom_wrapper.sitk_img.GetOrigin()
        # use the Trigger Time tag within one 
        time = dicom_wrapper.get_metadata_maybe('0018|1060')
        if sort_for_time:
            return (float(time), origin[2], origin[1], origin[0])
        else:
            return (origin[2], origin[1], origin[0])
    
    sorted_dicom_images = sorted(dicom_images, key=origin_sort)
    
    return sorted_dicom_images
    

In [9]:
def get_timesteps(dicom_images):
    """
    calculate the timesteps of one volume by summing up the origins
    all slices with same origin represents the timesteps of one slice
    """
    
    origins = [img.sitk_img.GetOrigin() for idx, img in enumerate(dicom_images)]
    counter = Counter(origins)
    steps = counter[next(iter(counter))]
    return steps

In [10]:
def get_contour(sitk_img, contour_dict, mixed_mode=False, mask=False):
    
    """
    tries to find a contour object for this dicom slice
    writes contours/masks into dicom slice
    
    sitk_img = sitk-image
    contour_dict = a flat dictionary with {uid : [contours]}
    class Contour():
        self.uid = uid
        self.image_size = image_size
        self.pixel_size = pixel_size
        self.points = points
        self.sub_pixel_res = sub_pixel_res
        self.tag = tag
    

    mixed_mode = True --> Use image slice as background
    mixed_mode = False --> Use emtpy slice as background
    
    mask = True --> fill contour, create masks
    mask = False --> Use plain contours
    
    returns (modified) dicom slice
    """

    # values to use for contour drawing
    tags = {
        'sarvepicardialContour': 0,
        'sarvendocardialContour': 1,
        'saepicardialContour' : 2,
        'saendocardialContour': 3,
    }
    
    # check if this slice SOP-UID is in the contour dictionary
    # check if contour is given for this slice
    contour = contour_dict.get(sitk_img.GetMetaData('0008|0018'), False)
    
    if contour: # if there is a contour for this slice given contour will be a list of contour objects,
        # if no contour is given for this slice contour will be False
        
        # go through contours in reverse order, to draw first the outer line, than inner
        contour = reversed(contour) # rvepi, rvendo, epi, endo
        
        # which backround should be used for this slice
        if mixed_mode: # use image slice as background   
            nda = sitk.GetArrayFromImage(sitk_img)
        else: # use empty slice as background
            temp_nda = sitk.GetArrayFromImage(sitk_img)
            shape = temp_nda.shape
            dtype = temp_nda.dtype
            nda = np.zeros(shape, dtype=dtype)
        
        # create mask or contour slice
        if mask: # create masks with given tag values
            nda_ = nda[0,:,:]
            for cont in contour:
                cv2.fillPoly(nda_, pts =[np.array(cont.points)], color=tags[cont.tag])
                nda = nda_[np.newaxis, :, :]
        
        else: # create contour points with given tag-values
            for cont in contour:
                for point in cont.points:
                    nda[0, point[1], point[0]] = tags[cont.tag]
            
        contour_img = sitk.GetImageFromArray(nda)

    
    else: # no contour given
        
        if mixed_mode: # mixed mode = True --> return image slice
            contour_img = sitk_img
            
        else: # mixed_mode = False --> return empty slice   
            temp_nda = sitk.GetArrayFromImage(sitk_img)
            nda = np.zeros(temp_nda.shape, dtype=temp_nda.dtype)
            contour_img = sitk.GetImageFromArray(nda)
        
    return contour_img
            

In [11]:
def get_valid_dicom_image(contours, args):
    
    # get a list of all series ids
    logging.info('----------------    Search {} series-ids for dicom directory ----------------')
    series_ids = FileWalker(GetDicomSeriesIDs, src_dir=args['src'], dest_dir=args['dest'], args=contours).open_files
    logging.info('----------------    Found {} series-ids in dicom directory ----------------'.format(len(set(series_ids))))
      
    # load and filter all dicom images with series-id in seried_ids as dicom_wrapper-classes
    logging.info('----------------    Load all dicom images with given series-ids {} -----------------')
    dicom_images = FileWalker(GetSegmenationDicoms, src_dir=args['src'], dest_dir=args['dest'], args=series_ids).open_files
    logging.info('----------------    All dicoms loaded: {} -----------------'.format(len(dicom_images)))
    return dicom_images

In [28]:
def build_volume(contour_as_dict, dicoms_sorted, args):
    
    """
    Builds 3 x 4D nrrd volumes:
    - dicom images sorted
    - contours sorted
    - masks sorted
    """
    
    image_slices = []
    image_slices_clean = []
    contour_slices = []
    mask_slices = []
    
    def get_metadata_maybe(sitk_img, key, default='not_found'):
        # helper for unicode decode errors
        try:
            value = sitk_img.GetMetaData(key)
        except Exception as e:
            logging.debug('key not found: {}, {}'.format(key, e))
            value = default
        # need to encode/decode all values because of unicode errors in the dataset
        if not isinstance(value, int):
            value = value.encode('utf8','backslashreplace').decode('utf-8').replace('\\udcfc','ue')
        return value
    
    timesteps = int(get_timesteps(dicoms_sorted))
    slices = int(len(dicoms_sorted)/timesteps)
    logging.info('Building volumes for patient: {}'.format(args['patient']))
    logging.info('images: {}'.format(len(dicoms_sorted)))
    logging.info('timesteps: {}'.format(timesteps))
    logging.info('slices: {}'.format(slices))
    lower_boundary = 0
    upper_boundary = slices
    
    spacing_3d = dicoms_sorted[0].sitk_img.GetSpacing()
    # build new spacing with t = 1
    z_spacing = spacing_3d[2]
    if z_spacing == 1: # some volumes dont have z spacing, they have one series per slice, use the slice thickness
        z_spacing = int(get_metadata_maybe(dicoms_sorted[0].sitk_img, '0018|0050', 6)) # default is 6
        logging.info('no spacing given, use slice thickness: {} as z-spacing ')
    spacing = (spacing_3d[0], spacing_3d[1], z_spacing, 1)
    # create as many 3d volume as timesteps
    for idx in range(timesteps):
        # need to sort every volumelist again, 
        # not all timesteps are the same within one timestep
        image_volume_aslist = dicoms_sorted[lower_boundary:upper_boundary]
        image_volume_aslist = sort_dicoms(image_volume_aslist, sort_for_time=False)

        # extract sitk images from dicom wrapper
        sitk_image_volume_aslist = [img.sitk_img for img in  image_volume_aslist]
        # contours
        contour_volume_aslist = [get_contour(img, contour_as_dict, mixed_mode=False, mask=False) for img in sitk_image_volume_aslist]
        # masks
        mask_volume_aslist = [get_contour(img, contour_as_dict, mixed_mode=False, mask=True) for img in sitk_image_volume_aslist]
        

        # extract numpy array and stack
        # images
        image_nda = np.stack([sitk.GetArrayFromImage(img) for img in sitk_image_volume_aslist], axis=1)
        # contours
        contour_nda = np.stack([sitk.GetArrayFromImage(img) for img in contour_volume_aslist], axis=1)
        # masks
        masks_nda = np.stack([sitk.GetArrayFromImage(img) for img in mask_volume_aslist], axis=1)

        
        # reshape 4d to 3d
        # images
        shape = image_nda.shape
        image_nda_reshaped = np.reshape(image_nda, (shape[1], shape[2], shape[3]))
        # contours
        shape = contour_nda.shape
        contour_nda_reshaped = np.reshape(contour_nda, (shape[1], shape[2], shape[3]))
        # masks
        shape = masks_nda.shape
        mask_nda_reshaped = np.reshape(masks_nda, (shape[1], shape[2], shape[3]))
        
        # append to list and increase boundaries counter
        image_slices_clean.append(sitk.GetImageFromArray(image_nda_reshaped))
        contour_slices.append(sitk.GetImageFromArray(contour_nda_reshaped))
        mask_slices.append(sitk.GetImageFromArray(mask_nda_reshaped))
        
        lower_boundary = lower_boundary + slices
        upper_boundary = upper_boundary + slices

    # join 3dvolumes to 4d volume
    # image
    new_img_clean = sitk.JoinSeries(image_slices_clean)
    
    # contour
    new_contour = sitk.JoinSeries(contour_slices)
    
    # mask
    new_mask = sitk.JoinSeries(mask_slices)
    

    size = new_img_clean.GetSize()
    dimension = new_img_clean.GetDimension()
    logging.info('volumes created for {}'.format(args['xml_file']))
    logging.info( "Image size: {}".format(size) )
    logging.info( "Image dimension: {}".format(dimension ))
    logging.info( "Image Spacing: {}".format(spacing ))
    logging.info('Writing images ...')
    

    # anyway, copy image information to new volume
    sitk_img = dicoms_sorted[0].sitk_img
    for tag in sitk_img.GetMetaDataKeys():

        value = get_metadata_maybe(sitk_img, tag)
        new_img_clean.SetMetaData(tag, value)
        new_contour.SetMetaData(tag, value)
        new_mask.SetMetaData(tag, value)
    
    new_img_clean.SetSpacing(spacing)
    new_contour.SetSpacing(spacing)
    new_mask.SetSpacing(spacing)

    sitk.WriteImage( new_img_clean, args['out_file_clean'] )
    logging.info('Image: {} done'.format( args['out_file_clean'] ))
    sitk.WriteImage( new_contour, args['out_file_c'] )
    logging.info('Image: {} done'.format( args['out_file_c'] ))
    sitk.WriteImage( new_mask, args['out_file_m'] )
    logging.info('Image: {} done'.format( args['out_file_m'] ))
    
    logging.info('Patient: {} done!'.format(args['patient']))
    
    
    

In [29]:

def convert_xml_to_nrrd(args):
    
    """
    Entry-point for one extraction, extract contours from one xml file
    and maps them to the corresponding dicom images
    """
    
    path = './reports/'

    # open xml file and create soup
    with open(args['xml_file']) as fp:
        soup = BeautifulSoup(fp, 'lxml')

    # load all segmentation contours
    contour_as_dict = extract_contours(soup)
    logging.info('Converted: {} contours'.format(len(contour_as_dict)))
    
    # load related dicom images
    dicom_images = get_valid_dicom_image(contours=contour_as_dict, args=args)
    dicoms_sorted = sort_dicoms(dicom_images)
    
    # build 4D-Volumes and write them to file system
    build_volume(contour_as_dict, dicoms_sorted, args)

    

In [30]:
def build_config(xml_file, dicom_folder, path_to_export):
    """
    build a dictionary with all directory paths for one Contour-Volume matching
    """
    
    
    f_name_clean = 'volume_clean.nrrd'
    f_name_segment = 'volume_contour.nrrd'
    f_name_mask = 'volume_mask.nrrd'
    dicom_patient = os.path.split(dicom_folder)[1]
    args = {}
    args['patient'] = dicom_patient
    args['xml_file'] = xml_file
    args['src'] = dicom_folder
    args['dest'] = os.path.join(path_to_export, dicom_patient)
    args['out_file_clean'] = os.path.join(path_to_export, dicom_patient + "_" + f_name_clean)
    args['out_file_c'] = os.path.join(path_to_export, dicom_patient + "_" + f_name_segment)
    args['out_file_m'] = os.path.join(path_to_export, dicom_patient + "_" + f_name_mask)
    
    logging.info('Build config for:')
    for key,value in args.items():
        logging.info('{}: {}'.format(key, value))
    
    return args

In [31]:
def map_xml_to_study(xml, dicom_study_folders):
    
    """
    extracts patient id etc. from xml file name
    transforms the format and search for that expression in all dicom folders
    """
    # parse patient uid, date from xml file and format it
    xml_file = os.path.basename(xml)
    patient_uid = xml_file[:8]
    logging.debug(patient_uid)
    study_date = xml_file[8:17]
    study_date = study_date[0:4] + "-" + study_date[4:6] + "-" + study_date[6:8]
    logging.debug(study_date)
    
    # match xml file and dicom folder according to the patient-UID and the study date
    matched = [folder for folder in dicom_study_folders if patient_uid in folder and study_date in folder]
    logging.debug(len(matched))
    if matched:
        # if there are two matches, return only the first
        logging.info('Found dicom folder: {} for XML file: {}.'.format(matched[0], xml_file))
        matched = matched[0]
    else :
        logging.info('Found NO dicom folder for XML file: {}.'.format(xml_file))
        logging.info('Searched for extracted patient-id: {} and date: {}'.format(patient_uid, study_date))
        matched = None
    # return anyway to track the the matched and missmatched
    return xml, matched

In [32]:
def create_volumes(path_to_xml, path_to_dicom, path_to_export):
    
    """
    Loads all XML files in path_to_xml
    Lists all patients folders in path_to_dicom_folders
    maps the xml file to patient dicom folder
    start contour extraction and dicom filter, sort and 4d building
    """
    
    xml_extension = '.cvi42wsx'
    
    # get all xml files
    if os.path.isdir(path_to_xml):
        
        xml_files = [os.path.join(path_to_xml, f) for f in os.listdir(path_to_xml) if os.path.isfile(os.path.join(path_to_xml, f))]
        # filter only cvi42wsx files
        xml_files = [file for file in xml_files if os.path.splitext(file)[1] == xml_extension]
        logging.info('Found {} xml-files.'.format(len(xml_files)))
        logging.debug(xml_files)
    else:
        logging.info('Error with path_to_xml: {}'.format(path_to_xml))
        return
    
    # get all study-folders
    if os.path.isdir(path_to_dicom):

        dicom_study_folders = [os.path.join(path_to_dicom, f) for f in os.listdir(path_to_dicom) if os.path.isdir(os.path.join(path_to_dicom, f))]
        logging.info('found {} Patient-Folders.'.format(len(dicom_study_folders)))
        logging.debug(dicom_study_folders)
        
    else:
        logging.info('Error with path_to_dicom: {}'.format(path_to_dicom))
        return
    
    # ensure export path exists
    ensure_dir(path_to_export)
    
    # get a list of (xml-files, study-folder) where we have a segmentation file and a dicom folder
    segmented_studies = [map_xml_to_study(xml, dicom_study_folders) for xml in xml_files]
    mapped_studies = [(xml, folder) for xml, folder in segmented_studies if folder is not None]
    ignored_files = [xml for xml, folder in segmented_studies if folder is None]
    logging.info('Matched: {} xml files to a dicom folder'.format(len(mapped_studies)))
    logging.debug(mapped_studies)
    logging.info('Didn\'t find a dicom folder for: {} xml files'.format(len(ignored_files)))
    [logging.info('No matching for: {}'.format(file)) for file in ignored_files]
    
    # extract contours, filter dicoms, sort slices and build 4d volumes
    patient = 0
    for xml_f, folder_n in mapped_studies:
        patient = patient + 1
        try:
            logging.info('processing patient: {} of total: {}'.format(patient, len(mapped_studies)))
            args = build_config(xml_f, folder_n, path_to_export)
            convert_xml_to_nrrd(args)
        except Exception as e:
            logging.error('Exception {} - Failed with xml: {} and dicom folder: {}'.format(str(e), str(xml_f), str(folder_n)))

    
    

In [ ]:
working_dir = os.getcwd()
path_to_xml = os.path.join(working_dir, 'data\\processed\\Segmentations\\xml_files\\')
path_to_dicom = 'E:\\sven\\ahf_export\\'
path_to_export = os.path.join(working_dir, 'data\\processed\\nrrd_volume_export_10\\')

create_volumes(path_to_xml, path_to_dicom, path_to_export)




2019-03-29 15:09:44,061 INFO Found 212 xml-files.
2019-03-29 15:09:44,125 INFO found 720 Patient-Folders.
2019-03-29 15:09:44,126 INFO Found dicom folder: E:\sven\ahf_export\0003-04NEJQUZ-2007-03-13 for XML file: 04NEJQUZ200703130733401.3.46.670589.11.0.0.11.4.2.0.8365.5.7500.2007031307333781538.cvi42wsx.
2019-03-29 15:09:44,127 INFO Found dicom folder: E:\sven\ahf_export\0006-0AE4R74L-1900-01-01 for XML file: 0AE4R74L190001011238291.3.12.2.1107.5.99.2.1013.30000008030612131778100072179.cvi42wsx.
2019-03-29 15:09:44,129 INFO Found dicom folder: E:\sven\ahf_export\0007-0HQQW4ZN-2007-05-23 for XML file: 0HQQW4ZN200705231338441.2.124.113532.192.168.202.14.20070523.131344.1689214.cvi42wsx.
2019-03-29 15:09:44,130 INFO Found dicom folder: E:\sven\ahf_export\0008-0PTV75MP-2005-06-27 for XML file: 0PTV75MP200506271208231.3.46.670589.11.0.0.11.4.2.0.5167.5.5240.2005062712082160343.cvi42wsx.
2019-03-29 15:09:44,131 INFO Found dicom folder: E:\sven\ahf_export\0010-0RPELLU8-2007-02-13 for XML fil

In [64]:
from sklearn.model_selection import train_test_split
import pandas as pd

# splitting with shuffle and seed
# first split train / test, then split train / validation

images = glob.glob(os.path.join(path_to_export, '*_clean.nrrd'))
masks = glob.glob(os.path.join(path_to_export, '*_mask.nrrd'))

seed = 42
X_train, X_test, Y_train, Y_test = train_test_split(images, masks, test_size=0.25, random_state=seed)
X_val, X_test, Y_val, Y_test = train_test_split(X_test, Y_test, test_size=0.5, random_state=seed)

print('4D Dataset split')
print('Dataset: images: {}, masks: {} '.format(len(images), len(masks)))
print('X_train: {}, Y_train: {}'.format(len(X_train), len(Y_train)))
print('X_val: {}, Y_val: {}'.format(len(X_val), len(Y_val)))
print('X_test: {}, Y_test: {}'.format(len(X_test), len(Y_test)))

4D Dataset split
Dataset: images: 201, masks: 201 
X_train: 150, Y_train: 150
X_val: 25, Y_val: 25
X_test: 26, Y_test: 26


In [65]:
# split dataset, define paths, set splitted datasets
IMPORT = pd.DataFrame(
    index=[
        'import_img', 'import_mask', 'img_train', 'img_val', 'img_test',
        'mask_train', 'mask_val', 'mask_test'
    ],
    columns=['Paths', 'Files', 'Data', 'Copied'])

IMPORT['Paths'] = [
    path_to_export, path_to_export,
    'data/processed/train/images/', 'data/processed/val/images/',
    'data/processed/test/images/', 'data/processed/train/masks/',
    'data/processed/val/masks/', 'data/processed/test/masks/'
    
]
IMPORT['Files'] = [
    len(images),
    len(masks),
    len(X_train),
    len(X_val),
    len(X_test),
    len(Y_train),
    len(Y_val),
    len(Y_test)
]

IMPORT['Data'] = [
    images,
    masks,
    X_train,
    X_val,
    X_test,
    Y_train,
    Y_val,
    Y_test
]


IMPORT

,Paths,Files,Data,Copied
import_img,G:\sven\git\cardio\data\processed\nrrd_volume_...,201,[G:\sven\git\cardio\data\processed\nrrd_volume...,NaN
import_mask,G:\sven\git\cardio\data\processed\nrrd_volume_...,201,[G:\sven\git\cardio\data\processed\nrrd_volume...,NaN
img_train,data/processed/train/images/,150,[G:\sven\git\cardio\data\processed\nrrd_volume...,NaN
img_val,data/processed/val/images/,25,[G:\sven\git\cardio\data\processed\nrrd_volume...,NaN
img_test,data/processed/test/images/,26,[G:\sven\git\cardio\data\processed\nrrd_volume...,NaN
mask_train,data/processed/train/masks/,150,[G:\sven\git\cardio\data\processed\nrrd_volume...,NaN
mask_val,data/processed/val/masks/,25,[G:\sven\git\cardio\data\processed\nrrd_volume...,NaN
mask_test,data/processed/test/masks/,26,[G:\sven\git\cardio\data\processed\nrrd_volume...,NaN


In [68]:
# define a helper copy function
def copy_volumes(files, destination_path):
    """
    Copy all files named in files to the destination path
    files: list of full file paths as strings
    destination_path: root destination folder as string
    """
    import shutil
    
    logging.info('Start to copy: {} files'.format(len(files)))
    # ensure directory exist
    ensure_dir(destination_path)
    
    for file in files:
        logging.info('copy file: {} to {}'.format(file, destination_path))
        shutil.copy2(file, destination_path)
    
    # get all nrrd files in the destination directory
    copied_files = glob.glob(os.path.join(destination_path, '*.nrrd'))
    
    logging.info('Copy finished, found {} files in destination path {}'.format(len(copied_files), destination_path))
    IMPORT.loc[IMPORT['Paths'] == destination_path] = len(copied_files)


In [69]:

# slice the dataframe
files_to_copy = IMPORT[['Paths', 'Data']][2:8]
# copy all data
[copy_volumes(row['Data'], row['Paths']) for index, row in files_to_copy.iterrows()]



2019-02-18 16:39:45,965 INFO Start to copy: 150 files
2019-02-18 16:39:45,969 INFO copy file: G:\sven\git\cardio\data\processed\nrrd_volume_export_9\0518-Q6RQ3Z87-2006-05-18_volume_clean.nrrd to data/processed/train/images/
2019-02-18 16:39:46,433 INFO copy file: G:\sven\git\cardio\data\processed\nrrd_volume_export_9\0013-13JLP3HN-2007-05-22_volume_clean.nrrd to data/processed/train/images/
2019-02-18 16:39:47,328 INFO copy file: G:\sven\git\cardio\data\processed\nrrd_volume_export_9\0345-GLKJL3AY-2006-12-14_volume_clean.nrrd to data/processed/train/images/
2019-02-18 16:39:47,351 INFO copy file: G:\sven\git\cardio\data\processed\nrrd_volume_export_9\0307-ER5Q2EXD-2007-02-22_volume_clean.nrrd to data/processed/train/images/
2019-02-18 16:39:47,585 INFO copy file: G:\sven\git\cardio\data\processed\nrrd_volume_export_9\0203-A3RJF3PQ-2005-09-22_volume_clean.nrrd to data/processed/train/images/
2019-02-18 16:39:47,948 INFO copy file: G:\sven\git\cardio\data\processed\nrrd_volume_export_9\0

[None, None, None, None, None, None]

In [70]:
IMPORT

,Paths,Files,Data,Copied
import_img,G:\sven\git\cardio\data\processed\nrrd_volume_...,201,[G:\sven\git\cardio\data\processed\nrrd_volume...,NaN
import_mask,G:\sven\git\cardio\data\processed\nrrd_volume_...,201,[G:\sven\git\cardio\data\processed\nrrd_volume...,NaN
img_train,150,150,150,150
img_val,25,25,25,25
img_test,26,26,26,26
mask_train,150,150,150,150
mask_val,25,25,25,25
mask_test,26,26,26,26
